## FSM with backup Kaggle data

In [50]:
from pyspark.sql import SparkSession
from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.recommendation import ALS

from pyspark.ml.evaluation import RegressionEvaluator

from pyspark.ml.tuning import CrossValidator, ParamGridBuilder

In [26]:
import os
import pandas as pd

In [10]:
os.getcwd()

'/Users/angelicadegaetano/Documents/Flatiron/Capstone/book_recommendation/notebooks/exploratory'

In [27]:

df_books = pd.read_csv('../../data/Kaggle_Dataset/books.csv')
df_ratings = pd.read_csv('../../data/Kaggle_Dataset/ratings.csv')
print(df_books.info())
print(df_ratings.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 23 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   id                         10000 non-null  int64  
 1   book_id                    10000 non-null  int64  
 2   best_book_id               10000 non-null  int64  
 3   work_id                    10000 non-null  int64  
 4   books_count                10000 non-null  int64  
 5   isbn                       9300 non-null   object 
 6   isbn13                     9415 non-null   float64
 7   authors                    10000 non-null  object 
 8   original_publication_year  9979 non-null   float64
 9   original_title             9415 non-null   object 
 10  title                      10000 non-null  object 
 11  language_code              8916 non-null   object 
 12  average_rating             10000 non-null  float64
 13  ratings_count              10000 non-null  int6

In [28]:
df= df_books.merge(df_ratings, left_on='book_id', right_on='book_id')
df.to_csv('../../data/Kaggle_Dataset/bookratings_merged.csv')

In [49]:
df.head(15)

,id,book_id,best_book_id,work_id,books_count,isbn,isbn13,authors,original_publication_year,original_title,...,work_text_reviews_count,ratings_1,ratings_2,ratings_3,ratings_4,ratings_5,image_url,small_image_url,user_id,rating
0,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,314,3
1,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,588,1
2,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,2077,2
3,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,2487,3
4,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,2900,3
5,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,3662,4
6,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,4536,1
7,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,5115,4
8,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,5272,5
9,2,3,3,4640799,491,439554934,9.780440e+12,"J.K. Rowling, Mary GrandPré",1997.0,Harry Potter and the Philosopher's Stone,...,75867,75504,101676,455024,1156318,3011543,https://images.gr-assets.com/books/1474154022m...,https://images.gr-assets.com/books/1474154022s...,5461,2


In [29]:
df2 = spark.read.csv('../../data/Kaggle_Dataset/bookratings_merged.csv', header='true', inferSchema='true')

In [30]:
df2.dtypes


[('_c0', 'int'),
 ('id', 'int'),
 ('book_id', 'int'),
 ('best_book_id', 'int'),
 ('work_id', 'int'),
 ('books_count', 'int'),
 ('isbn', 'string'),
 ('isbn13', 'double'),
 ('authors', 'string'),
 ('original_publication_year', 'double'),
 ('original_title', 'string'),
 ('title', 'string'),
 ('language_code', 'string'),
 ('average_rating', 'double'),
 ('ratings_count', 'int'),
 ('work_ratings_count', 'int'),
 ('work_text_reviews_count', 'int'),
 ('ratings_1', 'int'),
 ('ratings_2', 'int'),
 ('ratings_3', 'int'),
 ('ratings_4', 'int'),
 ('ratings_5', 'int'),
 ('image_url', 'string'),
 ('small_image_url', 'string'),
 ('user_id', 'int'),
 ('rating', 'int')]

In [47]:
df2.head()

Row(_c0=0, id=2, book_id=3, best_book_id=3, work_id=4640799, books_count=491, isbn='439554934', isbn13=9780439554930.0, authors='J.K. Rowling, Mary GrandPré', original_publication_year=1997.0, original_title="Harry Potter and the Philosopher's Stone", title="Harry Potter and the Sorcerer's Stone (Harry Potter, #1)", language_code='eng', average_rating=4.44, ratings_count=4602479, work_ratings_count=4800065, work_text_reviews_count=75867, ratings_1=75504, ratings_2=101676, ratings_3=455024, ratings_4=1156318, ratings_5=3011543, image_url='https://images.gr-assets.com/books/1474154022m/3.jpg', small_image_url='https://images.gr-assets.com/books/1474154022s/3.jpg', user_id=314, rating=3)

In [32]:
train, test = df2.randomSplit([0.8, 0.2])

In [34]:
als = ALS(maxIter=5,rank=4, regParam=0.01, userCol='user_id', itemCol='book_id', ratingCol='rating',
          coldStartStrategy='drop')


model = als.fit(train)

In [39]:
predictions = model.transform(test)
evaluator = RegressionEvaluator(metricName='rmse', labelCol='rating',
                                predictionCol='prediction')
rmse = evaluator.evaluate(predictions)
print('Root-mean-square error = ' + str(rmse))
#yikes rmse

Root-mean-square error = 5.348300880786995


In [42]:
als_model = ALS(userCol='user_id', itemCol='book_id', 
                ratingCol='rating', coldStartStrategy='drop')
                
params = ParamGridBuilder()\
          .addGrid(als_model.regParam, [0.01, 0.001])\
          .addGrid(als_model.rank, [4, 10]).build()


cv = CrossValidator(estimator=als_model, estimatorParamMaps=params,evaluator=evaluator,parallelism=4)
best_model = cv.fit(df2)    

best_model.bestModel.rank

10

In [44]:
users = df2.select(als.getUserCol()).distinct().limit(1)
userSubsetRecs = model.recommendForUserSubset(users, 10)
recs = userSubsetRecs.take(1)
recs

[Row(user_id=19984, recommendations=[Row(book_id=1618, rating=15.685933113098145), Row(book_id=9646, rating=13.4002046585083), Row(book_id=7062, rating=12.838693618774414), Row(book_id=6870, rating=12.434889793395996), Row(book_id=9517, rating=12.241125106811523), Row(book_id=7081, rating=12.165765762329102), Row(book_id=3882, rating=11.345526695251465), Row(book_id=6420, rating=11.062800407409668), Row(book_id=3483, rating=10.984026908874512), Row(book_id=8172, rating=10.972114562988281)])]

In [46]:
first_recommendation = recs[0]['recommendations'][0][0]

first_recommendation

1618